In [49]:
from sklearn.model_selection import RepeatedStratifiedKFold
from ipynb.fs.full.data_wrangling import * #Data preprocessing notebook
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import statistics


SCORINGS = {
    'f1': make_scorer(f1_score, average = None),
    'precision': make_scorer(precision_score, average = None),
    'recall': make_scorer(recall_score, average = None),
    'roc_auc': make_scorer(roc_auc_score, average = None)
}

cv_splits = 10
repetitions = 1
RANDOM_STATE = 42
cross_validation_setting = RepeatedStratifiedKFold(n_splits=cv_splits,
                                                   n_repeats=repetitions,
                                                   random_state= RANDOM_STATE)
        
def cross_validate(model, X_train, X_test, y_train, y_test, metric):
    for index in range(len(X_train)):
        model.fit(X_train, y_train)
        Y_pred = model.predict(X_test)


def model_evaluation(model, features, target, 
                     cv = cross_validation_setting):
    scores = dict()
    formatted_scores = dict()
    formatted_scores['model'] = model
    for scoring_name, scoring_function in SCORINGS.items():
        scores[score_metric] = cross_validate(model, X, Y, 
                                              scoring = scoring_function,
                                              cv = cross_validation_setting)
        return scores[score_metric]
#         formatted_scores[score_metric + " score"] = scores[score_metric].mean()
#         formatted_scores[score_metric + " std"] = scores[score_metric].std()
#     return formatted_scores

def get_scores(Y_pred, Y_true):
    f1 = f1_score(Y_true, Y_pred, average=None)
    precision = precision_score(Y_true, Y_pred,
                                average = None)
    recall = recall_score(Y_true, Y_pred, 
                          average = None)
    roc = roc_auc_score(Y_true, Y_pred, 
                        average = None)
    return f1, precision, recall, roc

def convert_df(X, Y):
    return X.to_numpy(), Y.to_numpy()
    
def evaluate_model(model, X, Y, sk_fold):
    X, Y = convert_df(X, Y)
    f1_list, precision_list, recall_list, auc_list = [], [], [], []
    
    for train_index, test_index in sk_fold.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
    
        f1, precision, recall, roc = get_scores(Y_pred, Y_test)
        f1_list.append(f1)
        precision_list.append(precision)
        recall_list.append(recall)
        auc_list.append(roc)
        
    return format_return(f1_list, precision_list,
                         recall_list, auc_list)

COLUMNS = ["F1 Average", "F1 Class 0", "F1 Class 1",
           "Recall Average", "Recall Class 0", "Recall Class 1",
           "AUC Average", "AUC Class 0", "AUC Class 1",
           "Precision Average", "Precision Class 0", "Precision Class 1"]

def format_return(f1_list, precision_list,
                  recall_list, auc_list):
    dataframe = pd.DataFrame(columns = COLUMNS)
    dataframe_line = []
    aux = []
    lists_of_score_list = []
    lists_of_score_list.append(f1_list)
    lists_of_score_list.append(precision_list)
    lists_of_score_list.append(recall_list)
    lists_of_score_list.append(auc_list)
    for score_list in lists_of_score_list:
        dataframe_line = dataframe.append(statistics.mean(flatten_list(score_list)))
        dataframe_line = dataframe.append(statistics.mean([score[0] for score in score_list]))
        dataframe_line = dataframe.append(statistics.mean([score[1] for score in score_list]))
    dataframe.append(pd.DataFrame(dataframe_line, columns = COLUMNS), ignore_index = True)
    return dataframe

def flatten_list(lista):
    return [value for sublist in lista for value in sublist]
    
    

    
        
        
        


    
    
    

In [50]:
"""
Config for every experiment
"""
RANDOM_STATE = 0
VOTING_METHOD = 'soft'
results = pd.DataFrame(columns = ['model', "accuracy score", "accuracy std"])


In [51]:
"""
Getting the best weak learners (GridSearchCV section)
"""
from sklearn.model_selection import GridSearchCV
# --------------------------------------------

In [52]:
"""
K Neighbors
"""
from sklearn.neighbors import KNeighborsClassifier

k_range = list(range(1,31))
weight_options = ["uniform", "distance"]

knn_grid = dict(n_neighbors = k_range, 
                weights = weight_options)
knn = KNeighborsClassifier()
knn_gridsearch = GridSearchCV(knn, param_grid = knn_grid,
                             refit = True)
selected_knn = knn_gridsearch.fit(X, Y).best_estimator_
a,b,c = evaluate_model(selected_knn, X, Y, cross_validation_setting)
#results = results.append(model_evaluation(selected_knn, X , Y), ignore_index = True)

AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
a

In [ ]:
print(flatten_list([[0,1], [2,3]]))

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

log_reg_grid={'C':[0.001,0.01,.09,1,5,10],
              "penalty":["l1","l2"]} #l1 lasso l2 ridge
lr = LogisticRegression(random_state=RANDOM_STATE)
log_reg_gridsearch = GridSearchCV(lr, param_grid = log_reg_grid,
                             refit = True)
selected_log_reg = log_reg_gridsearch.fit(X,Y).best_estimator_
evaluate_model(selected_log_reg, X.to_numpy(), Y.to_numpy())
#results = results.append(model_evaluation(selected_log_reg, X, Y), ignore_index = True)

In [ ]:
"""
Support Vector Machines
"""
from sklearn.svm import SVC

svm_grid = {'C': [0.1, 1, 10, 100, 1000], 
            'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
            'kernel': ['rbf']} 
  
svm_gridsearch = GridSearchCV(SVC(),
                              param_grid = svm_grid,
                              refit = True)

selected_svm = svm_gridsearch.fit(X,Y).best_estimator_
model_evaluation(selected_svm, X , Y)

In [ ]:
results

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

nb_params = {'var_smoothing': np.logspace(0,-9, num=100)}
nb_gridsearch = GridSearchCV(estimator=nb, 
                     param_grid = nb_params,
                     refit = True)
selected_nb = nb_gridsearch.fit(X,Y).best_estimator_
model_evaluation(selected_nb, X, Y)


In [ ]:
"""Boosting Algorithms"""
from sklearn.ensemble import AdaBoostClassifier
boosting_models = dict()

In [ ]:
#AdaBoost
ab_param_grid = {
    'n_estimators' : [100, 300, 500],
    'learning_rate' : [1e-3, 1e-2, 1e-1, 1]
}
ab_model = AdaBoostClassifier(random_state = RANDOM_STATE)
xgb_gridsearch = GridSearchCV(ab_model,
                              param_grid = ab_param_grid,
                              refit = True)

selected_xgb = xgb_gridsearch.fit(X, Y).best_estimator_
model_evaluation(selected_xgb, X, Y)

In [ ]:
#XGBoost
import xgboost as xgb

parameters = {
            'max_depth': [3, 4, 5],
            'learning_rate': [0.01, 0.1, 1],
            'n_estimators': [200, 400],
            'gamma': [0.01, 0.1, 0.2],
            'min_child_weight': [0, 0.5, 1],
            'max_delta_step': [0],
            'subsample': [0.7, 1],
            'colsample_bytree': [0.6, 1],
            'reg_alpha': [0, 1e-2, 1],
            'reg_lambda': [0, 1e-2, 1],
            }

xgb_model = xgb.XGBClassifier(silent = True,
                              random_state = RANDOM_STATE)

xgb_gridsearch = GridSearchCV(xgb_model,
                              parameters,
                              refit = True)

selected_xgb = xgb_gridsearch.fit(X, Y).best_estimator_
model_evaluation(selected_xgb, X, Y)

In [ ]:
"""Bagging Algorithms"""
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bagging_models = dict()
bagging_models['rf'] = RandomForestClassifier(random_state = RANDOM_STATE)
bagging_models['et'] = ExtraTreesClassifier(random_state = RANDOM_STATE)
bagging_models['bdt'] = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                          random_state = RANDOM_STATE)


In [ ]:
#RandomForest
rf_grid = {
    'max_depth': [4, 5, 6],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [200, 400]
}

rf = RandomForestClassifier(random_state = RANDOM_STATE)
rf_gridsearch = GridSearchCV(rf, param_grid = rf_grid,
                             refit = True)
selected_rf = rf_gridsearch.fit(X, Y).best_estimator_
model_evaluation(selected_rf, X, Y)

In [ ]:
#Extra Trees

et_grid = {
    'max_depth': [4, 5, 6],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [200, 400],
    'oob_score': [True, False]
}

In [ ]:
"""Stacking Ensemble"""
from sklearn.ensemble import StackingClassifier

stacking_estimators = [
    ('lr', selected_lr),
    ('knn', selected_knn),
    ('svm', selected_svm),
    ('gnb', selected_nb),
    ('dt', selected_dt)
]

final_estimator = selected_lr

stacking_model = StackingClassifier(estimators = stacking_estimators,
                                    final_estimator = final_estimator)



In [ ]:
"""Voting Ensemble"""
from sklearn.ensemble import VotingClassifier

voting_estimators = [
    ('lr', LogisticRegression(random_state=RANDOM_STATE)),
    ('knn', KNeighborsClassifier()),
    ('svm', SVC(random_state=RANDOM_STATE)),
    ('gnb', GaussianNB()),
    ('dt', DecisionTreeClassifier(random_state=RANDOM_STATE))
]

voting_classifier = VotingClassifier(voting_estimators,
                                     voting=VOTING_METHOD)